In [2]:
import numpy as np
import cv2
from numpy import linalg

dir1 = "./images/bottle1.JPG"
dir2 = "./images/bottle2.JPG"

img1 = cv2.imread(dir1)
img2 = cv2.imread(dir2)

In [3]:
def get_points(img1,img2):
    # from each image select 8points
    points1 = []
    points2 = []

    def OnMouseAction1(event,x,y,flags,param):

        if event == cv2.EVENT_LBUTTONDOWN:
            print("from camera1:(%s,%s)"%(x,y))
            points1.append([x,y])
        elif event==cv2.EVENT_RBUTTONDOWN :
            print("camera1 finish")
            cv2.destroyWindow("image1")
            
    def OnMouseAction2(event,x,y,flags,param):

        if event == cv2.EVENT_LBUTTONDOWN:
            print("from camera2:(%s,%s)"%(x,y))
            points2.append([x,y])
        elif event==cv2.EVENT_RBUTTONDOWN :
            print("camera2 finish")
            cv2.destroyWindow("image2")


    cv2.namedWindow("image1",cv2.WINDOW_NORMAL)
    cv2.resizeWindow("image1", 540, 480)
    cv2.setMouseCallback('image1',OnMouseAction1)     
    cv2.namedWindow("image2",cv2.WINDOW_NORMAL)
    cv2.resizeWindow("image2", 540, 480)
    cv2.setMouseCallback('image2',OnMouseAction2)
    cv2.imshow('image1',img1)
    cv2.imshow('image2',img2)
    cv2.waitKey(80000)
    
         
    
    return (np.array(points1),np.array(points2))

In [4]:
def normalization(pts):
    # Normalizing the input points #
    # Find the centroid of the points (find the mean x and mean y value)
    centroid = np.array([np.mean(pts[:, 0]), np.mean(pts[:, 1])])

    # Compute the mean distance of all the points from this centroid
    distance = 0
    for index in range(pts[:, 0].size):
        distance += np.linalg.norm(centroid - pts[index, :])
    meanDistance = distance / pts[:, 0].size

    # Construct a 3 by 3 matrix that would translate the points so that the mean distance would be sqrt(2)
    T = np.array([[np.sqrt(2) / meanDistance, 0, -centroid[0] * np.sqrt(2) / meanDistance],
                  [0, np.sqrt(2) / meanDistance, -centroid[1] * np.sqrt(2) / meanDistance],
                  [0, 0, 1]])

    # Normalize points
    result = np.ones((pts[:, 0].size, 2), dtype=float)
    for index in range(pts[:, 0].size):
        current = np.dot(T, np.array([pts[index, 0], pts[index, 1], 1]))
        # we are only interested in x & y
        result[index, 0] = current[0]
        result[index, 1] = current[1]

    return result, T

In [ ]:
def calc_epipole(F):
    """ F行列からエピポールeを算出 """

    # Fを特異値分解
    U, D, V = linalg.svd(F)
    # Vから固有値最小の固有ベクトルを取り出す
    fvec = V[2]

    return fvec / fvec[2]


In [5]:
def FM_by_normalized_8_point(pts1, pts2):
    # F, _ = cv2.findFundamentalMat(pts1, pts2, cv2.FM_8POINT)
    # comment out the above line of code.

    # Normalizing points
    normalizedPts1, T1 = normalization(pts1)
    normalizedPts2, T2 = normalization(pts2)

    print(normalizedPts1)
    print(normalizedPts1.shape)

    u1 = normalizedPts1[:, 0]
    v1 = normalizedPts1[:, 1]
    u2 = normalizedPts2[:, 0]
    v2 = normalizedPts2[:, 1]
    ones = np.ones(normalizedPts1[:, 0].size, int)

    # Create the constraint matrix
    A = np.array([u1 * u2, u1 * v2, u1, v1 * u2, v1 * v2, v1, u2, v2, ones]).transpose()
    print(A.shape)
    U, D, V = np.linalg.svd(A)

    # Reference: https://www.coursera.org/lecture/robotics-perception/epipolar-geometry-ii-WRyoL #
    # Extract fundamental matrix from the column of V corresponding to the smallest singular value
    FMatrix = np.reshape(V.transpose()[:, 8], (3, 3)).transpose()
    print(FMatrix)
    # Enforce rank2 constraint
    U, D, V = np.linalg.svd(FMatrix)
    FMatrix = U @ np.diag(np.array([D[0], D[1], 0])) @ V
    print(FMatrix)
    # De-normalize
    FMatrix = T2.transpose() @ FMatrix @ T1
    print(FMatrix)
    ##############################################################################################

    # normalize the fundamental matrix so that the value on [2, 2] is 1
    FMatrix = np.true_divide(FMatrix, FMatrix[2, 2])
    print(FMatrix.shape)
    print(pts1.shape)
    
    _,eig_list_e1 = linalg.eig(np.dot(FMatrix.T,FMatrix))
    e1 = eig_list_e1.T[-1]/eig_list_e1.T[-1][2]
    _,eig_list_e2 = linalg.eig(np.dot(FMatrix,FMatrix.T))
    e2 = eig_list_e2.T[-1]/eig_list_e2.T[-1][2]
    
    return (FMatrix, e1, e2)

In [6]:
def draw_lines(point1,point2,e1,e2,dir1,dir2):
    one = np.ones((len(point1),1),"float")
    point1_ = np.concatenate((point1, one),axis=1)
    point2_ = np.concatenate((point2, one),axis=1)
    line1 = []
    line2 = []
    i = 0
    j = 0
    
    for p1 , p2 in zip(point1_,point2_):
        line1.append(np.cross(e1, p1))
        line2.append(np.cross(e2, p2))
        
    
    cv2.namedWindow('epilines1',cv2.WINDOW_AUTOSIZE)
    img1 = cv2.imread(dir1)
    for l1,p1 in zip(line1,point1):
        i += 1
        aa = tuple(p1.tolist())
        print("circle")
        cv2.circle(img1, aa, 15, (255,0,16), -1)
        cv2.putText(img1,'%s'%i,(p1[0],p1[1]+10),cv2.FONT_HERSHEY_COMPLEX,3,(0,0,255),9)
        cv2.line(img1, (15, int(-l1[2] / l1[1])),(img1.shape[1], int(-(l1[2] + l1[0] * img1.shape[1]) / l1[1])),(0,255,0),5)
    cv2.imshow('epilines1',img1)
    cv2.imwrite("./result/2-1_lines.jpg", img1)
    cv2.waitKey(0)
    
    
    cv2.namedWindow('epilines2',cv2.WINDOW_AUTOSIZE)
    img2 = cv2.imread(dir2)
    for l2,p2 in zip(line2,point2):
        j += 1
        bb = tuple(p2.tolist())
        cv2.circle(img2, bb, 15, (255,0,16), -1)
        cv2.putText(img2,'%s'%j,(p2[0],p2[1]+10),cv2.FONT_HERSHEY_COMPLEX,3,(0,0,255),9)
        cv2.line(img2, (0, int(-l2[2] / l2[1])),(img2.shape[1], int(-(l2[2] + l2[0] * img2.shape[1]) / l2[1])), (0,255,0),5)
    cv2.imshow('epilines2',img2)
    cv2.imwrite("./result/2-2_lines.jpg", img2)
    cv2.waitKey(0)

In [7]:
points1,points2 = get_points(img1,img2)
print(points1)
print(points2)


from camera1:(299,1903)
from camera2:(500,2054)
from camera1:(1008,296)
from camera2:(1142,346)
from camera1:(343,2432)
from camera2:(553,2614)
from camera1:(1396,2230)
from camera2:(1590,2293)
from camera1:(732,850)
from camera2:(911,914)
from camera1:(3957,1128)
from camera2:(3927,1109)
from camera1:(3853,2236)
from camera2:(3868,2123)
from camera1:(963,2520)
from camera2:(1254,2633)
camera1 finish
camera2 finish
[[ 299 1903]
 [1008  296]
 [ 343 2432]
 [1396 2230]
 [ 732  850]
 [3957 1128]
 [3853 2236]
 [ 963 2520]]
[[ 500 2054]
 [1142  346]
 [ 553 2614]
 [1590 2293]
 [ 911  914]
 [3927 1109]
 [3868 2123]
 [1254 2633]]


In [14]:
F, e1, e2 = FM_by_normalized_8_point(points1,points2)

print(F)
print(e1, e2)

[[-1.21776096  0.19526849]
 [-0.53785741 -1.34578229]
 [-1.17556667  0.70255901]
 [-0.16578043  0.50884883]
 [-0.80253073 -0.81451774]
 [ 2.29011942 -0.5479265 ]
 [ 2.19038745  0.5146026 ]
 [-0.58101067  0.7869476 ]]
(8, 2)
(8, 9)
[[ 0.00553407 -0.10337977  0.06789294]
 [ 0.13372211  0.00379     0.68451432]
 [-0.02381789 -0.70541417  0.00634581]]
[[ 0.00930778 -0.10351688  0.06715859]
 [ 0.13335715  0.00380326  0.68458535]
 [-0.02437366 -0.70539398  0.00645396]]
[[ 8.84804121e-09 -9.84039201e-08  2.19917237e-04]
 [ 1.26770300e-07  3.61540956e-09  4.73591011e-04]
 [-2.61786238e-04 -5.13741116e-04 -1.90621797e-02]]
(3, 3)
(8, 2)
[[-4.64167338e-07  5.16225960e-06 -1.15368358e-02]
 [-6.65035697e-06 -1.89664016e-07 -2.48445361e-02]
 [ 1.37332793e-02  2.69508065e-02  1.00000000e+00]]
[-14.79088834 -29.59545443   1.        ] [15.60267296 33.04177914  1.        ]


In [15]:
F2, _ = cv2.findFundamentalMat(points1, points2, cv2.FM_8POINT)

In [16]:
print(F2)

[[-4.64167338e-07  5.16225960e-06 -1.15368358e-02]
 [-6.65035697e-06 -1.89664016e-07 -2.48445361e-02]
 [ 1.37332793e-02  2.69508065e-02  1.00000000e+00]]


In [18]:
lines1 = cv2.computeCorrespondEpilines(points2.reshape(-1, 1, 2), 1, F)
lines1 = lines1.reshape(-1, 3)

print(lines1)

[[-4.0780734e-02 -9.9916810e-01  2.2119253e+03]
 [-3.0156019e-01 -9.5344716e-01  7.6288940e+02]
 [ 5.8505591e-02 -9.9828708e-01  2.7192988e+03]
 [-1.2209908e-02 -9.9992543e-01  2.3603774e+03]
 [-2.2693940e-01 -9.7390890e-01  1.1954048e+03]
 [-1.4756635e-01 -9.8905218e-01  1.6394194e+03]
 [-4.6500985e-02 -9.9891824e-01  2.1819651e+03]
 [ 4.3698601e-02 -9.9904478e-01  2.6451497e+03]]


In [12]:
# draw_lines(points1,points2,e1,e2,dir1,dir2)

In [17]:
lines1 = cv2.computeCorrespondEpilines(points2.reshape(-1, 1, 2), 2, F)
lines1 = lines1.reshape(-1, 3)


lines2 = cv2.computeCorrespondEpilines(points1.reshape(-1, 1, 2), 1, F)
lines2 = lines2.reshape(-1, 3)


def drawlines_new(img1, img2, lines, pts1, pts2):
    ''' img1 - image on which we draw the epilines for the points in img2
        lines - corresponding epilines '''
    r = img1.shape[0]
    c = img1.shape[1]
    i = 0
    # img1 = cv2.cvtColor(img1, cv2.COLOR_RGB2GRAY)
    # img2 = cv2.cvtColor(img2, cv2.COLOR_RGB2GRAY)
    for r, pt1, pt2 in zip(lines, pts1, pts2):
        i += 1
        color = tuple(np.random.randint(0, 255, 3).tolist())
        x0, y0 = map(int, [0, -r[2] / r[1]])
        x1, y1 = map(int, [c, -(r[2] + r[0] * c) / r[1]])
        cv2.putText(img1, '%s' % i, (pt1[0], pt1[1]+10), cv2.FONT_HERSHEY_COMPLEX, 3, (0, 0, 255), 9)
        img1 = cv2.line(img1, (x0, y0), (x1, y1), color, 4)
        img1 = cv2.circle(img1, tuple(pt1), 12, color, -1)
        img2 = cv2.circle(img2, tuple(pt2), 12, color, -1)
    return img1, img2


line_img1, line_img2 = drawlines_new(img1, img2, lines1, points1, points2)
line_img3, line_img4 = drawlines_new(img2, img1, lines2, points2, points1)

cv2.imwrite("./result/lines1.jpg", line_img1)
# cv2.imwrite("./result/lines2.jpg", line_img2)
cv2.imwrite("./result/lines2.jpg", line_img3)
# cv2.imwrite("./result/lines4.jpg", line_img4)


True

In [ ]:
U,sigma,VT = linalg.svd(F)
sigma_mat = np.array([[sigma[0],0,0],[0,sigma[1],0],[0,0,0]])
F_2 = np.dot(np.dot(U,sigma_mat),VT)